In [5]:
import schnetpack as spk
import torch
import schnetpack.nn 
import schnetpack.data
import pandas as pd
import scipy.linalg as la
from schnetpack.datasets import QM9






In [6]:
#Intermediare embedding output pipeline

#Load last checkpoint file from from trained model
Checkpoint = './trained_models/qm911/checkpoints/checkpoint-5.pth.tar'

#Load the Best model where it is stored after training
best_model = torch.load('./trained_models/qm911/best_model')
model = best_model

model.load_state_dict(torch.load(Checkpoint), strict=False)

# Download QM9 dataset to use in evaluating model
qm9data = QM9('./qm9.db', download=True, remove_uncharacterized=True)

# Load split file for the train, validation and test 
train, val, test = spk.data.train_test_split(qm9data, split_file='./trained_models/qm911/split.npz')





#set up device and atoms converter for input
device = 'cpu'

converter = spk.data.AtomsConverter(device=device)

test_loader = spk.AtomsLoader(test, batch_size=100)
converter = spk.data.AtomsConverter(device=device)

at, props = qm9data.get_properties(idx=1)

calculator = spk.interfaces.SpkCalculator(model=best_model, device=device, energy=QM9.U0)
at.set_calculator(calculator)


inputs = converter(at)

print('Prediction:', at.get_total_energy())


Prediction: -1541.551025390625


In [9]:

#We choose None to instatiate the variable originally
embedding_output=None
#Define a hooking function that will fetch the out_tensor 
#from the layer of interest given an input tensor
def embedding_hook(self, inp_tensor, out_tensor):
    # Self is included and refers to the model class
    # Global allows us to utilize embedding_output outside the current function scope
    global embedding_output
    #Update the embedding_output variable to be equal to our output tensor
    embedding_output=out_tensor

#Hook our function to the embedding layer during the forward pass
model.representation.embedding.register_forward_hook(embedding_hook)

#Check if None
print(embedding_output)

#Forward pass of the tensor inputs
model(inputs)

#embedding_output SHOULD be changed
print(embedding_output)

None
tensor([[[-1.1789e+00,  1.0051e+00,  1.2346e-02, -4.9465e-01, -2.7075e+00,
           1.2768e+00,  7.0167e-01, -8.4085e-01,  9.3975e-01,  3.7902e-01,
           4.6136e-01,  1.1211e+00, -3.2832e-01,  1.5644e+00,  2.0077e+00,
           2.2989e-01, -1.2679e-01, -7.0355e-01,  5.1581e-01, -1.2680e+00,
           2.2348e+00, -9.1419e-01,  1.1213e-01,  8.3197e-01, -1.4527e+00,
           4.2544e-01,  1.3293e-01,  1.4175e-01,  1.0613e+00,  1.8534e+00],
         [ 2.4999e+00, -8.8243e-02, -2.7747e-01,  1.0349e+00, -1.5068e-02,
           1.4981e+00,  4.7207e-01,  1.2192e+00,  7.4943e-02, -1.3184e+00,
           2.2984e-01,  1.2548e+00,  7.9738e-01,  1.5308e+00, -1.5439e+00,
           1.3754e+00,  9.1261e-01, -2.3110e+00, -7.7784e-01,  7.5336e-01,
           6.1621e-01, -6.4225e-02,  2.0356e-04, -6.9536e-02,  1.2833e+00,
           1.0686e+00,  2.2942e-01, -6.9096e-01, -1.8814e-01, -1.3653e+00],
         [ 2.4999e+00, -8.8243e-02, -2.7747e-01,  1.0349e+00, -1.5068e-02,
           1.4981e

In [14]:

interactions_output=None

def interactions_hook(self, inp_tensor, out_tensor):
    global interactions_output
    interactions_output=out_tensor

model.representation.interactions.register_forward_hook(interactions_hook)

#Check if None
print(interactions_output)

#Forward pass of the tensor inputs
model(inputs)

#embedding_output SHOULD be changed
print(interactions_output)

None
None


In [ ]:

#We choose None to instatiate the variable originally
embedding_output=None
#Define a hooking function that will fetch the out_tensor 
#from the layer of interest given an input tensor
def embedding_hook(self, inp_tensor, out_tensor):
    # Self is included and refers to the model class
    # Global allows us to utilize embedding_output outside the current function scope
    global embedding_output
    #Update the embedding_output variable to be equal to our output tensor
    embedding_output=out_tensor

#Hook our function to the embedding layer during the forward pass
model.representation.embedding.register_forward_hook(embedding_hook)

#Check if None
print(embedding_output)

#Forward pass of the tensor inputs
model(inputs)

#embedding_output SHOULD be changed
print(embedding_output)

In [ ]:
"""
1) Write hook functions for each layer in the model and compile a database of the intermediare outputs.


2) Store these outputs into a h5 file using h5py
    How does h5 work?
        - create a dictionary of dataset
            dataset = h5py.File("name.h5", "w")
            dataset.create_dataset("name_dataset", data=np.array)
            ...
            dataset.close()
            dataset["name_dataset"]

3) Small toy systems of interest where we can look at specific chemical groups
    3.1a) Carboxylic acids
    3.2a) Ketone
    3.3a) Aldehyde
    3.4a) Alcohols
    3.5a) Ethers
    
    3.1b) Amine
    3.2b) Amide
    3.3b) Azo
    3.4b) Nitro
    ...
    
    Contrast chemically similar groups that are distinguished by atom type
    
4) Construct descriptors to differentiate between different chemical functionalities
    4.1) PCA
    4.2) t-SNE
    4.3) Self-Organising maps
    4.4) Autoencoders
"""